In [ ]:
# ============================================================
# Environment Setup
# ============================================================

import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = os.path.abspath('../..')
if project_root not in sys.path:
    sys.path.append(project_root)

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import re

# NLP imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, f1_score
)

# SageMaker imports
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.sklearn import SKLearnModel

# Configuration
try:
    from utils.sagemaker_config import get_sagemaker_config
    config = get_sagemaker_config(s3_prefix='lab4-sentiment')
    role = config['role']
    session = config['session']
    bucket = config['bucket']
    region = config['region']
except ImportError:
    print("Using fallback configuration")
    role = get_execution_role()
    session = sagemaker.Session()
    bucket = session.default_bucket()
    region = session.boto_region_name

print("Configuration complete.")
print(f"Region: {region}")
print(f"S3 Bucket: {bucket}")

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## ⚠️ Endpoint Cleanup (Run if getting scaler.pkl error)

Si vous obtenez l'erreur `scaler.pkl not found`, exécutez cette cellule pour supprimer les anciens endpoints avant de redéployer.

In [ ]:
# ============================================================
# Cleanup Old Endpoints (if needed)
# ============================================================

import boto3

print("🔍 Checking for old sentiment-analysis endpoints...")
print("=" * 60)

sm_client = boto3.client('sagemaker')

try:
    # List existing endpoints
    response = sm_client.list_endpoints(
        NameContains='sentiment-analysis',
        StatusEquals='InService'
    )
    
    endpoints = response['Endpoints']
    
    if not endpoints:
        print("✅ No old endpoints found. You can proceed with deployment.")
    else:
        print(f"\n⚠️  Found {len(endpoints)} existing endpoint(s):\n")
        for ep in endpoints:
            print(f"  • {ep['EndpointName']} (Created: {ep['CreationTime']})")
        
        print("\n🗑️  Deleting old endpoints to avoid scaler.pkl errors...")
        for ep in endpoints:
            endpoint_name = ep['EndpointName']
            sm_client.delete_endpoint(EndpointName=endpoint_name)
            print(f"✅ Deleted: {endpoint_name}")
        
        print("\n✅ Cleanup complete! You can now deploy a new endpoint.")
        
except Exception as e:
    print(f"Note: {e}")
    print("Proceeding with notebook...")

---

## Section 1: Data Generation and Exploration

We'll generate synthetic product reviews with varying sentiments.

### Sentiment Categories

| Sentiment | Score Range | Characteristics |
|-----------|-------------|-----------------|
| Positive | 4-5 stars | Praise, satisfaction |
| Neutral | 3 stars | Mixed feelings, OK |
| Negative | 1-2 stars | Complaints, dissatisfaction |


In [ ]:
# ============================================================
# Generate Synthetic Review Dataset
# ============================================================

def generate_reviews(n_samples=1500, random_state=42):
    """Generate synthetic product reviews"""
    np.random.seed(random_state)
    
    # Review templates by sentiment
    templates = {
        'positive': [
            "Excellent product! {}",
            "Love this! {}",
            "Great quality, {}",
            "Highly recommend! {}",
            "Perfect! {}",
            "Amazing, {}",
            "Best purchase ever! {}",
            "Outstanding {}",
            "Fantastic product, {}",
            "Exceeded expectations! {}"
        ],
        'neutral': [
            "It's okay, {}",
            "Average product, {}",
            "Nothing special, {}",
            "Decent but {}",
            "Works as expected, {}",
            "Acceptable, {}",
            "Fair quality, {}",
            "So-so, {}",
            "Not bad, {}",
            "Could be better, {}"
        ],
        'negative': [
            "Terrible! {}",
            "Very disappointed, {}",
            "Poor quality, {}",
            "Waste of money! {}",
            "Don't buy this, {}",
            "Awful product, {}",
            "Regret buying, {}",
            "Horrible, {}",
            "Not as described, {}",
            "Complete disaster! {}"
        ]
    }
    
    # Detail phrases
    details = {
        'positive': [
            "works perfectly",
            "great value for money",
            "fast shipping",
            "exactly as described",
            "will buy again",
            "looks great",
            "high quality materials",
            "easy to use",
            "durable and reliable",
            "customer service was excellent"
        ],
        'neutral': [
            "does the job",
            "average quality",
            "price is fair",
            "shipping took a while",
            "meets basic needs",
            "nothing extraordinary",
            "standard quality",
            "acceptable for the price",
            "got what I paid for",
            "works but could improve"
        ],
        'negative': [
            "broke after one day",
            "not worth the price",
            "terrible quality",
            "doesn't work as advertised",
            "waste of time",
            "poor customer service",
            "damaged on arrival",
            "stopped working quickly",
            "misleading description",
            "requesting refund"
        ]
    }
    
    reviews = []
    sentiments = []
    ratings = []
    
    # Generate balanced dataset
    samples_per_class = n_samples // 3
    
    for sentiment in ['positive', 'neutral', 'negative']:
        for _ in range(samples_per_class):
            template = np.random.choice(templates[sentiment])
            detail = np.random.choice(details[sentiment])
            
            review = template.format(detail)
            
            # Add variation
            if np.random.random() < 0.2:
                review = review.upper()
            
            # Assign rating based on sentiment
            if sentiment == 'positive':
                rating = np.random.choice([4, 5], p=[0.3, 0.7])
            elif sentiment == 'neutral':
                rating = 3
            else:
                rating = np.random.choice([1, 2], p=[0.6, 0.4])
            
            reviews.append(review)
            sentiments.append(sentiment)
            ratings.append(rating)
    
    # Create DataFrame
    df = pd.DataFrame({
        'review_id': [f'REV_{i:06d}' for i in range(len(reviews))],
        'review_text': reviews,
        'sentiment': sentiments,
        'rating': ratings
    })
    
    # Shuffle
    df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    return df

# Generate dataset
print("Generating product review dataset...")
reviews_df = generate_reviews(n_samples=1500, random_state=42)

print(f"\nDataset shape: {reviews_df.shape}")
print(f"\nSentiment distribution:")
print(reviews_df['sentiment'].value_counts())
print(f"\nRating distribution:")
print(reviews_df['rating'].value_counts().sort_index())
print(f"\nSample reviews:")
reviews_df.head(10)

In [ ]:
# ============================================================
# Text Preprocessing
# ============================================================

def preprocess_review(text):
    """Clean review text"""
    text = text.lower()
    text = re.sub(r'!+', '!', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

reviews_df['review_clean'] = reviews_df['review_text'].apply(preprocess_review)

# Text statistics
reviews_df['text_length'] = reviews_df['review_clean'].str.len()
reviews_df['word_count'] = reviews_df['review_clean'].str.split().str.len()

print("Text Statistics by Sentiment:")
print("="*60)
print(reviews_df.groupby('sentiment')[['text_length', 'word_count']].describe())

---

## Section 2: Exploratory Data Analysis


In [ ]:
# ============================================================
# Visualization
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sentiment distribution
ax = axes[0, 0]
reviews_df['sentiment'].value_counts().plot(kind='bar', ax=ax, color=['green', 'gray', 'red'])
ax.set_title('Sentiment Distribution')
ax.set_xlabel('Sentiment')
ax.set_ylabel('Count')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

# Rating distribution
ax = axes[0, 1]
reviews_df['rating'].value_counts().sort_index().plot(kind='bar', ax=ax, color='skyblue')
ax.set_title('Rating Distribution')
ax.set_xlabel('Rating (stars)')
ax.set_ylabel('Count')

# Text length by sentiment
ax = axes[1, 0]
reviews_df.boxplot(column='text_length', by='sentiment', ax=ax)
ax.set_title('Text Length by Sentiment')
ax.set_xlabel('Sentiment')
ax.set_ylabel('Characters')

# Word count by sentiment
ax = axes[1, 1]
reviews_df.boxplot(column='word_count', by='sentiment', ax=ax)
ax.set_title('Word Count by Sentiment')
ax.set_xlabel('Sentiment')
ax.set_ylabel('Words')

plt.tight_layout()
plt.show()

# Correlation between rating and sentiment
print("\nRating vs Sentiment Crosstab:")
print(pd.crosstab(reviews_df['rating'], reviews_df['sentiment']))

---

## Section 3: Feature Extraction and Model Training

Using TF-IDF for cost-effective sentiment analysis.


In [ ]:
# ============================================================
# TF-IDF Feature Extraction
# ============================================================

# Prepare data
X_text = reviews_df['review_clean']
y = reviews_df['sentiment']

# Label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Label mapping:")
for idx, label in enumerate(label_encoder.classes_):
    print(f"  {label}: {idx}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_text, y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

print(f"\nTraining samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# TF-IDF Vectorization
tfidf = TfidfVectorizer(
    max_features=800,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8,
    stop_words='english'
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(f"\nTF-IDF matrix shape: {X_train_tfidf.shape}")
print(f"Vocabulary size: {len(tfidf.vocabulary_)}")

# Top words per sentiment
print("\nTop 10 words per sentiment:")
print("="*60)

feature_names = tfidf.get_feature_names_out()
for idx, sentiment in enumerate(label_encoder.classes_):
    sentiment_mask = y_train == idx
    sentiment_tfidf = X_train_tfidf[sentiment_mask].mean(axis=0).A1
    top_indices = sentiment_tfidf.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_indices]
    print(f"\n{sentiment.upper()}:")
    print(f"  {', '.join(top_words)}")

In [ ]:
# ============================================================
# Train Sentiment Classifier
# ============================================================

# Train logistic regression with balanced classes
model = LogisticRegression(
    random_state=42,
    max_iter=500,
    class_weight='balanced',
    multi_class='multinomial',
    solver='lbfgs'
)

print("Training sentiment classifier...")
model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = model.predict(X_test_tfidf)
y_pred_proba = model.predict_proba(X_test_tfidf)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print(f"\nModel Performance:")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  F1 Score (macro): {f1_macro:.4f}")
print(f"  F1 Score (weighted): {f1_weighted:.4f}")

In [ ]:
# ============================================================
# Detailed Evaluation
# ============================================================

print("Classification Report:")
print("="*60)
print(classification_report(
    y_test,
    y_pred,
    target_names=label_encoder.classes_
))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.title('Sentiment Analysis Confusion Matrix')
plt.ylabel('True Sentiment')
plt.xlabel('Predicted Sentiment')
plt.show()

# Per-sentiment accuracy
print("\nPer-Sentiment Accuracy:")
for idx, sentiment in enumerate(label_encoder.classes_):
    mask = y_test == idx
    sentiment_acc = accuracy_score(y_test[mask], y_pred[mask])
    print(f"  {sentiment.capitalize()}: {sentiment_acc:.4f}")

---

## Section 4: Model Interpretation and Testing


In [ ]:
# ============================================================
# Feature Importance
# ============================================================

print("Most Important Words per Sentiment:")
print("="*60)

feature_names = tfidf.get_feature_names_out()

for idx, sentiment in enumerate(label_encoder.classes_):
    coef = model.coef_[idx]
    top_positive_idx = coef.argsort()[-10:][::-1]
    top_words = [(feature_names[i], coef[i]) for i in top_positive_idx]
    
    print(f"\n{sentiment.upper()} - Top indicators:")
    for word, score in top_words:
        print(f"  {word}: {score:.4f}")

# Test with sample reviews
print("\n\nSample Predictions:")
print("="*60)

test_reviews = [
    "Excellent product! Works perfectly and great value",
    "It's okay, does the job but nothing special",
    "Terrible quality! Broke after one day, waste of money",
    "Amazing! Highly recommend this to everyone",
    "Not bad but could be better for the price"
]

for review in test_reviews:
    clean_review = preprocess_review(review)
    review_tfidf = tfidf.transform([clean_review])
    
    pred_idx = model.predict(review_tfidf)[0]
    pred_proba = model.predict_proba(review_tfidf)[0]
    pred_sentiment = label_encoder.classes_[pred_idx]
    
    print(f"\nReview: '{review}'")
    print(f"Predicted: {pred_sentiment.upper()} (confidence: {pred_proba[pred_idx]:.2%})")
    print(f"All probabilities: {dict(zip(label_encoder.classes_, pred_proba))}")

---

## Section 5: Model Deployment

Deploy sentiment analysis model to SageMaker.


In [ ]:
# ============================================================
# Package Model for Deployment
# ============================================================

import joblib
import tarfile
import shutil

# Create model directory
model_dir = 'sentiment_model'
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)
os.makedirs(model_dir)

# Save artifacts
joblib.dump(model, os.path.join(model_dir, 'model.pkl'))
joblib.dump(tfidf, os.path.join(model_dir, 'tfidf.pkl'))
joblib.dump(label_encoder, os.path.join(model_dir, 'label_encoder.pkl'))

print("Model artifacts saved")

# Create inference script
inference_code = """
import joblib
import os
import json
import re

def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, 'model.pkl'))
    tfidf = joblib.load(os.path.join(model_dir, 'tfidf.pkl'))
    label_encoder = joblib.load(os.path.join(model_dir, 'label_encoder.pkl'))
    return {'model': model, 'tfidf': tfidf, 'label_encoder': label_encoder}

def preprocess_review(text):
    text = text.lower()
    text = re.sub(r'!+', '!', text)
    text = re.sub(r'\\s+', ' ', text)
    return text.strip()

def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        data = json.loads(request_body)
        if isinstance(data, dict):
            return [data.get('review', '')]
        elif isinstance(data, list):
            return [item.get('review', '') if isinstance(item, dict) else str(item) for item in data]
        else:
            return [str(data)]
    else:
        return [request_body]

def predict_fn(input_data, model_dict):
    model = model_dict['model']
    tfidf = model_dict['tfidf']
    label_encoder = model_dict['label_encoder']
    
    # Preprocess
    reviews_clean = [preprocess_review(review) for review in input_data]
    
    # Vectorize
    X_tfidf = tfidf.transform(reviews_clean)
    
    # Predict
    predictions = model.predict(X_tfidf)
    probabilities = model.predict_proba(X_tfidf)
    
    # Format results
    results = []
    for pred_idx, proba in zip(predictions, probabilities):
        sentiment = label_encoder.classes_[pred_idx]
        confidence = float(proba[pred_idx])
        all_probs = {label: float(prob) for label, prob in zip(label_encoder.classes_, proba)}
        
        results.append({
            'sentiment': sentiment,
            'confidence': confidence,
            'probabilities': all_probs
        })
    
    return results

def output_fn(prediction, accept):
    if accept == 'application/json':
        return json.dumps(prediction), accept
    raise ValueError(f'Unsupported accept type: {accept}')
"""

with open(os.path.join(model_dir, 'inference.py'), 'w') as f:
    f.write(inference_code)

# Create requirements
requirements = """scikit-learn==1.3.0
joblib==1.3.2
numpy==1.24.3
"""

with open(os.path.join(model_dir, 'requirements.txt'), 'w') as f:
    f.write(requirements)

# Create tar.gz
model_archive = 'model.tar.gz'
with tarfile.open(model_archive, 'w:gz') as tar:
    tar.add(model_dir, arcname='.')

# Upload to S3
model_s3_key = f'lab4-sentiment/models/{datetime.now().strftime("%Y%m%d-%H%M%S")}/model.tar.gz'
s3_client = boto3.client('s3')
s3_client.upload_file(model_archive, bucket, model_s3_key)

model_s3_uri = f's3://{bucket}/{model_s3_key}'
print(f"Model uploaded to: {model_s3_uri}")

In [ ]:
# ============================================================
# Deploy to SageMaker Endpoint
# ============================================================

sklearn_model = SKLearnModel(
    model_data=model_s3_uri,
    role=role,
    entry_point='inference.py',
    framework_version='1.2-1',
    py_version='py3',
    name=f'sentiment-analysis-{datetime.now().strftime("%Y%m%d-%H%M%S")}',
    env={'SAGEMAKER_PROGRAM': 'inference.py'}
)

print("Deploying model to endpoint...")
print("This will take 5-10 minutes...")

predictor = sklearn_model.deploy(

In [ ]:
# ============================================================
# Test Endpoint
# ============================================================

test_reviews = [
    {"review": "Absolutely love this product! Best purchase ever"},
    {"review": "It's decent, works as expected"},
    {"review": "Terrible quality, very disappointed"},
    {"review": "Amazing! Exceeded all my expectations"}
]

print("Testing sentiment analysis endpoint:")
print("="*60)

for review in test_reviews:
    response = predictor.predict(review)
    
    print(f"\nReview: '{review['review']}'")
    print(f"Sentiment: {response[0]['sentiment'].upper()}")
    print(f"Confidence: {response[0]['confidence']:.2%}")

# Batch prediction
print("\n\nBatch prediction test:")
batch_response = predictor.predict(test_reviews)

for i, (review, result) in enumerate(zip(test_reviews, batch_response)):
    print(f"\n{i+1}. {result['sentiment'].upper()} ({result['confidence']:.0%}) - '{review['review'][:50]}...'")

---

## Section 8: SageMaker Model Monitor - Data & Model Drift


In [ ]:
# ============================================================
# SageMaker Model Monitor - Setup
# ============================================================

from sagemaker.model_monitor import DefaultModelMonitor, CronExpressionGenerator
from sagemaker.model_monitor.dataset_format import DatasetFormat

print("SageMaker Model Monitor - Setup")
print("=" * 60)
print("\nModel Monitor tracks data quality and model performance drift\n")

# Créer un Model Monitor
model_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=session
)

print(f"Model Monitor created: {model_monitor}")

In [ ]:
# ============================================================
# Create Baseline Dataset
# ============================================================

print("\nCreating baseline dataset for monitoring...")
print("-" * 60)

# Préparer des données de baseline (échantillon du train set)
baseline_df = pd.DataFrame({
    'review': train_reviews[:1000],
    'sentiment': [label_encoder.inverse_transform([s])[0] for s in train_labels[:1000]]
})

# Sauvegarder le baseline
baseline_path = f's3://{bucket}/model-monitor/baseline'
baseline_file = '/tmp/baseline.csv'
baseline_df.to_csv(baseline_file, index=False, header=False)
boto3.client('s3').upload_file(baseline_file, bucket, 'model-monitor/baseline/baseline.csv')

print(f"Baseline dataset saved: {baseline_path}")
print(f"Baseline size: {len(baseline_df)} samples")

# Suggérer une baseline (requires endpoint data capture enabled)
print("\nNote: In production, you would:")
print("1. Enable data capture on the endpoint")
print("2. Suggest baseline from captured data")
print("3. Create monitoring schedule")
print("\nExample code:")
print("""
# Enable data capture when deploying
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f's3://{bucket}/data-capture'
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    data_capture_config=data_capture_config
)

# Suggest baseline
model_monitor.suggest_baseline(
    baseline_dataset=baseline_path,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=f's3://{bucket}/model-monitor/baseline-results'
)
""")

In [ ]:
# ============================================================
# Monitor Data Quality - Simulated Drift
# ============================================================

print("\n\nSimulating Data Drift Detection:")
print("=" * 60)

# Simuler des données de production (avec drift)
print("\n1. Baseline Distribution (Training Data):")
baseline_sentiments = train_df['sentiment'].value_counts(normalize=True)
print(baseline_sentiments)

# Simuler du drift: plus de reviews négatives
drift_data = pd.concat([
    train_df[train_df['sentiment'] == 'negative'].sample(600, replace=True),
    train_df[train_df['sentiment'] == 'neutral'].sample(200, replace=True),
    train_df[train_df['sentiment'] == 'positive'].sample(200, replace=True)
])

print("\n2. Production Distribution (With Drift):")
drift_sentiments = drift_data['sentiment'].value_counts(normalize=True)
print(drift_sentiments)

# Calculer la divergence
from scipy.stats import chi2_contingency

baseline_counts = train_df['sentiment'].value_counts()
drift_counts = drift_data['sentiment'].value_counts()

# Aligner les indices
all_categories = list(set(baseline_counts.index) | set(drift_counts.index))
baseline_aligned = [baseline_counts.get(cat, 0) for cat in all_categories]
drift_aligned = [drift_counts.get(cat, 0) for cat in all_categories]

# Chi-square test
chi2, p_value, dof, expected = chi2_contingency([baseline_aligned, drift_aligned])

print(f"\n3. Drift Detection:")
print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("⚠️  ALERT: Significant distribution shift detected!")
    print("Action: Investigate cause and consider retraining")
else:
    print("✅ No significant drift detected")

# Visualiser le drift
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

baseline_sentiments.plot(kind='bar', ax=axes[0], color='skyblue', alpha=0.7)
axes[0].set_title('Baseline Distribution (Training)')
axes[0].set_ylabel('Proportion')
axes[0].set_xlabel('Sentiment')
axes[0].set_ylim(0, 1)
axes[0].grid(True, alpha=0.3)

drift_sentiments.plot(kind='bar', ax=axes[1], color='coral', alpha=0.7)
axes[1].set_title('Production Distribution (With Drift)')
axes[1].set_ylabel('Proportion')
axes[1].set_xlabel('Sentiment')
axes[1].set_ylim(0, 1)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model_monitor_drift.png', dpi=100, bbox_inches='tight')
print(f"\n📊 Drift visualization saved")
plt.show()

In [ ]:
# ============================================================
# Monitor Model Quality
# ============================================================

print("\n\nMonitoring Model Quality:")
print("=" * 60)

# Comparer performance sur baseline vs drift data
print("\n1. Baseline Performance:")
baseline_pred = model.predict(vectorizer.transform(train_reviews[:1000]))
baseline_accuracy = accuracy_score(train_labels[:1000], baseline_pred)
print(f"Accuracy: {baseline_accuracy:.2%}")

print("\n2. Production Performance (with drift):")
drift_reviews = drift_data['review'].tolist()
drift_labels_encoded = label_encoder.transform(drift_data['sentiment'])
drift_pred = model.predict(vectorizer.transform(drift_reviews))
drift_accuracy = accuracy_score(drift_labels_encoded, drift_pred)
print(f"Accuracy: {drift_accuracy:.2%}")

# Calculer la dégradation
degradation = (baseline_accuracy - drift_accuracy) / baseline_accuracy * 100
print(f"\n3. Model Degradation: {degradation:.2f}%")

if abs(degradation) > 5:
    print("⚠️  ALERT: Model performance degraded significantly!")
    print("Action: Retrain model with recent data")
else:
    print("✅ Model performance stable")

# Classification report détaillé
print("\n4. Detailed Performance on Drift Data:")
print(classification_report(
    drift_labels_encoded, 
    drift_pred, 
    target_names=label_encoder.classes_
))

---

### SageMaker Model Monitor Benefits

**Data Quality Monitoring:**
- Detect input data drift
- Track feature distributions
- Identify missing or invalid values
- Alert on anomalous inputs

**Model Quality Monitoring:**
- Track prediction accuracy over time
- Detect performance degradation
- Monitor prediction drift
- Trigger retraining when needed

**Monitoring Schedule:**
```python
# In production, create a monitoring schedule
from sagemaker.model_monitor import CronExpressionGenerator

model_monitor.create_monitoring_schedule(
    monitor_schedule_name=f'sentiment-monitor-{timestamp}',
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f's3://{bucket}/monitoring-results',
    statistics=baseline_statistics_uri,
    constraints=baseline_constraints_uri,
    schedule_cron_expression=CronExpressionGenerator.hourly()
)
```

**Metrics to Monitor:**
- **Data Drift**: Distribution shifts, new categories
- **Model Drift**: Accuracy, precision, recall changes
- **Inference Patterns**: Request volume, latency
- **Data Quality**: Missing values, outliers

**Alerting Options:**
- CloudWatch Alarms
- SNS notifications
- Lambda triggers for auto-retraining
- Email/Slack alerts

**When to Retrain:**
- Significant data drift (p-value < 0.05)
- Performance degradation > 5%
- New patterns in production data
- Scheduled periodic retraining


In [ ]:
# ============================================================
# Generate Monitoring Report
# ============================================================

print("\n\nModel Monitoring Report:")
print("=" * 60)

monitoring_report = {
    'timestamp': datetime.now().isoformat(),
    'baseline_metrics': {
        'samples': len(train_df),
        'accuracy': float(baseline_accuracy),
        'distribution': baseline_sentiments.to_dict()
    },
    'production_metrics': {
        'samples': len(drift_data),
        'accuracy': float(drift_accuracy),
        'distribution': drift_sentiments.to_dict()
    },
    'drift_detection': {
        'chi2_statistic': float(chi2),
        'p_value': float(p_value),
        'significant_drift': bool(p_value < 0.05)
    },
    'model_health': {
        'degradation_percent': float(degradation),
        'action_required': bool(abs(degradation) > 5),
        'recommendation': 'Retrain model' if abs(degradation) > 5 else 'Monitor'
    }
}

print(json.dumps(monitoring_report, indent=2))

# Sauvegarder le rapport
report_path = '/tmp/monitoring_report.json'
with open(report_path, 'w') as f:
    json.dump(monitoring_report, f, indent=2)

# Upload vers S3
boto3.client('s3').upload_file(
    report_path, 
    bucket, 
    f'model-monitor/reports/{datetime.now().strftime("%Y%m%d-%H%M%S")}.json'
)

print(f"\n✅ Monitoring report saved to S3")
print(f"\nRecommendation: {monitoring_report['model_health']['recommendation']}")

In [ ]:
# ============================================================
# Cleanup All Resources
# ============================================================

print("🧹 Cleaning up Lab 4 resources...")
print("=" * 60)

# Delete ALL sentiment-analysis endpoints
sm_client = boto3.client('sagemaker')

try:
    response = sm_client.list_endpoints(NameContains='sentiment-analysis')
    endpoints = response['Endpoints']
    
    if endpoints:
        print(f"\n🗑️  Deleting {len(endpoints)} endpoint(s)...")
        for ep in endpoints:
            endpoint_name = ep['EndpointName']
            try:
                sm_client.delete_endpoint(EndpointName=endpoint_name)
                print(f"  ✅ Deleted: {endpoint_name}")
            except Exception as e:
                print(f"  ⚠️  {endpoint_name}: {e}")
    else:
        print("  No endpoints to delete")
except Exception as e:
    print(f"  Error listing endpoints: {e}")

# Cleanup local files
print("\n📁 Cleaning up local files...")
try:
    if 'model_dir' in globals() and os.path.exists(model_dir):
        shutil.rmtree(model_dir)
        print("  ✅ Deleted model directory")
    if 'model_archive' in globals() and os.path.exists(model_archive):
        os.remove(model_archive)
        print("  ✅ Deleted model archive")
except Exception as e:
    print(f"  ⚠️  {e}")

print("\n✅ Lab 4 cleanup complete!")
print("💡 Note: S3 data and models are kept for future use")

---

## Summary

In this lab, you:

1. Generated and analyzed synthetic product review data
2. Built sentiment classification model using TF-IDF
3. Evaluated model performance on 3-class sentiment task
4. Interpreted feature importance to understand predictions
5. Deployed sentiment model to SageMaker endpoint

### Key Takeaways

- **Sentiment Analysis**: Critical for understanding customer feedback
- **Multi-class Classification**: Neutral sentiment adds complexity
- **TF-IDF**: Cost-effective approach for sentiment tasks
- **Feature Importance**: Shows which words drive sentiment predictions
- **SageMaker Deployment**: Production-ready sentiment analysis

### Cost Optimization

| Component | Choice | Benefit |
|-----------|--------|---------|
| Model | TF-IDF + Logistic Regression | No GPU required |
| Instance | ml.m5.large | Cost-effective |
| Training | < 1 minute | Minimal cost |
| Vocabulary | 800 features | Small model size |

### Next Steps

- Lab 5: MLOps Packaging (Feature Store, Model Registry)
- Try different sentiment granularities (5-star ratings)
- Experiment with handling sarcasm and mixed sentiments
- Add aspect-based sentiment analysis

---

**Remember to delete your endpoint to avoid charges!**
